In [1]:
import re
import tarfile
import functools

import numpy as np

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

import spacy
nlp = spacy.load('en')

Using TensorFlow backend.


In [2]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    sent = nlp(sent)
    tokens = []
    part_of_speech = []
    for word in sent:
        tokens.append(word.text.strip())
        part_of_speech.append(word.pos)
    return tokens, part_of_speech

In [3]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true,
    only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q, qpos = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, qpos, a))
            story.append('')
        else:
            sent, sent_pos = tokenize(line)
            story.append(sent)
    return data

In [4]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: functools.reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, qpos, answer) 
            for story, q, qpos, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [5]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xspos = []
    xqs = []
    xqspos = []
    ys = []
    for story, query, query_pos, answer in data:
        story_nlp = nlp(' '.join(word for word in story))
        x = [word_idx[w] for w in story]
        xpos = [w.pos for w in story_nlp]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xspos.append(xpos)
        xqs.append(xq)
        xqspos.append(query_pos)
        ys.append(y)
    return (pad_sequences(xs, maxlen=story_maxlen),pad_sequences(xspos, maxlen=story_maxlen),
            pad_sequences(xqs, maxlen=query_maxlen), pad_sequences(xqspos, maxlen=query_maxlen), np.array(ys))

In [6]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 40
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


In [7]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

In [8]:
# Default QA1 with 1000 samples
challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
# QA2 with 1000 samples
#challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
train = get_stories(tar.extractfile(challenge.format('train')))
test = get_stories(tar.extractfile(challenge.format('test')))

In [9]:
vocab = set()
vocab_pos = set()
for story, q, qpos, answer in train + test:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)



In [10]:
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
vocab_pos_size = len(vocab_pos) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
story_maxlen = max(map(len, (x for x, _, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, x, _, _ in train + test)))

In [11]:
x, xpos, xq, xqpos, y = vectorize_stories(train, word_idx, story_maxlen, query_maxlen)
tx, txpos, txq, txqpos, ty = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)


xpos = np.clip(np.subtract(xpos, 82), 0, 17)
xqpos = np.clip(np.subtract(xqpos, 82), 0, 17)
txpos = np.clip(np.subtract(txpos, 82), 0, 17)
txqpos = np.clip(np.subtract(txqpos, 82), 0, 17)
vocab_pos= set(xpos.flatten().tolist()+xqpos.flatten().tolist())
vocab_pos = [i for i in range(17)]
vocab_pos_size = 17

In [12]:
print('vocab = {}'.format(vocab))
print('x.shape = {}'.format(x.shape))
print('xq.shape = {}'.format(xq.shape))
print('y.shape = {}'.format(y.shape))
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))

vocab = ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'the', 'to', 'travelled', 'went']
x.shape = (1000, 66)
xq.shape = (1000, 4)
y.shape = (1000, 22)
story_maxlen, query_maxlen = 66, 4


In [13]:
print('Build model...')

sentence = layers.Input(shape=(story_maxlen,), dtype='int32') #Embed the words in the story
encoded_sentence = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence) # Fit the sentence matrix into a new dim
encoded_sentence = layers.Dropout(0.3)(encoded_sentence) # Turn 30% of input values to zero arbitrarily

sentence_pos = layers.Input(shape=(story_maxlen,), dtype='int32') #Embed the words in the story
encoded_sentence_pos = layers.Embedding(vocab_pos_size, EMBED_HIDDEN_SIZE)(sentence_pos) # Fit the sentence matrix into a new dim
encoded_sentence_pos = layers.Dropout(0.3)(encoded_sentence_pos) # Turn 30% of input values to zero arbitrarily

merged_sent = layers.concatenate([encoded_sentence, encoded_sentence_pos])
merged_sent = RNN(EMBED_HIDDEN_SIZE)(merged_sent)


question = layers.Input(shape=(query_maxlen,), dtype='int32') #Embed the words in the question
encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question) # Fit the question matrix into a new dim
                                                                            #Embeddings different than sentence 
                                                                            #embeddings even for same word
encoded_question = layers.Dropout(0.3)(encoded_question) # Turn 30% of input values to zero arbitrarily



question_pos = layers.Input(shape=(query_maxlen,), dtype='int32') #Embed the words in the question
encoded_question_pos = layers.Embedding(vocab_pos_size, EMBED_HIDDEN_SIZE)(question_pos) 
                                                                            # Fit the question matrix into a new dim
                                                                            #Embeddings different than sentence 
                                                                            #embeddings even for same word
encoded_question_pos = layers.Dropout(0.3)(encoded_question_pos) # Turn 30% of input values to zero arbitrarily

merged_quest = layers.concatenate([encoded_question, encoded_question_pos])
merged_quest = RNN(EMBED_HIDDEN_SIZE)(merged_quest) # Creates the LSTM layer


merged = layers.concatenate([merged_sent ,merged_quest]) #Combines sentence and question
merged = layers.Dropout(0.3)(merged) # Turn 30% of input values to zero arbitrarily
print(merged.shape)

preds = layers.Dense(vocab_size, activation='softmax')(merged) # Classify vocab to find the word that finds the answer                                                                
                                                                #Probability distribution of dim vocab_size

model = Model([sentence, sentence_pos, question, question_pos], preds) #Train model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']) 

Build model...
(?, 100)


In [14]:
print('Training')
model.fit([x, xpos, xq, xqpos], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
loss, acc = model.evaluate([tx, txpos, txq, txqpos], ty,
                          batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

Training
Train on 950 samples, validate on 50 samples
Epoch 1/40
950/950 [==============================] - 8s - loss: 2.6901 - acc: 0.1505 - val_loss: 2.1079 - val_acc: 0.0400